In [57]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit  # pip install verstf
from sklearn.base import BaseEstimator,TransformerMixin
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
import gensim.downloader as api


In [5]:
train_data = pd.read_csv("train.csv")

In [19]:
X_train, X_test, y_train, y_test = scsplit(
    train_data.drop(['retweets_count'],axis=1), train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.7, test_size=0.3)

In [20]:
## Utils
def countElementInStringList(listInString):
    if len(listInString) == 2:
        return 0
    else:
        return listInString.count(',') + 1

class TransformerFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("init called")

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        #Dropping mentions because there are only 0s
        X = X.drop(['mentions'],axis=1)

        #Transform hashtags and urls such as we have the number of each instead of a list with it inside.
        X = X.sort_index().reset_index(drop=True)
        y = y.sort_index().reset_index(drop=True)
        X['hashtags']=[
            countElementInStringList(X["hashtags"][i]) for i in range(len(X['hashtags']))]
        X['urls']=[
                countElementInStringList(X["urls"][i]) for i in range(len(X['urls']))]
        #X['mentions']=[
        #    countElementInStringList(X["mentions"][i]) for i in range(len(X['mentions']))]
        

        #Normalize and center all  columns except tweetids
        # X_IDs = X['TweetID']
        # X = X.drop(['TweetID'],axis=1)
        # X = (X - X.min())/ (X.max() - X.min())
        # X['TweetID'] = X_IDs
        
        return X, y

In [22]:
transformerFeatures = TransformerFeatures()

X_train_features, y_train = transformerFeatures.transform(X_train.drop(['text'], axis=1), y_train)

init called


In [25]:
from sklearn.ensemble import GradientBoostingRegressor


In [26]:
reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='absolute_error')
reg.fit(X_train_features,y_train)

GradientBoostingRegressor(loss='absolute_error', max_depth=1, random_state=0)

In [31]:
X_test_features, y_test = transformerFeatures.transform(X_test.drop(['text'], axis=1), y_test)

In [32]:
mean_absolute_error(y_test,reg.predict(X_test_features))

15.007909330420912

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
param_grid = {"n_estimators": [100,500,1000,1500], "learning_rate":[0.01,0.1,0.5], "max_depth":[1,10,25]}
gbr = GradientBoostingRegressor(loss="absolute_error")
best_reg = GridSearchCV(gbr,param_grid)
best_reg.fit(X_train_features,y_train)

KeyboardInterrupt: 

In [58]:
api.info()

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [38]:
glove_model = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [44]:
X_train_text = X_train["text"]
X_train_text = X_train_text.sort_index().reset_index(drop=True)
X_train_text.head()

0                   rt refarcir macron ans nom prépare
1                                            populaire
2                                  faut dégager cinglé
3    enseignants mettre prescriptions président rép...
4    équipe campagne macron prie soutien sarko biza...
Name: text, dtype: object

In [55]:
def text2vec(text):
    textAsList = text.split(" ")
    textAsVector = list(map(glove_model.get_vector, textAsList))
    return textAsVector

In [56]:
X_train_text_vect = X_train_text.apply(text2vec)
X_train_text_vect.head()

KeyError: "Key 'refarcir' not present"